<font size="+2" color="blue">Additional results: experiments on cross-department association rules</font>

# Practice Session 04: Basket analysis

Author: <font color="blue">Marcel Fernández Serrano</font>

E-mail: <font color="blue">marcel.fernandez02@estudiant.upf.edu</font>

Date: <font color="blue">17/10/2023</font>

In [1]:
import numpy as np  
import matplotlib.pyplot as plt  
import pandas as pd  
import csv
import gzip
from apyori import apriori

# 1. Playing with apyori

In [2]:
# LEAVE AS-IS

def print_apyori_output (association_results, info=False, info_key=False):
    for relation_record in association_results:
        itemset = list(relation_record.items)
        
        # Consider only itemsets of two elements
        if len(itemset) > 1: 
        
            print("Rules involving itemset %s" % itemset)
            support = relation_record.support

            for rules in relation_record.ordered_statistics:
                antecedent = list(rules.items_base)
                consequent = list(rules.items_add)
                
                if info_key:
                    antecedent = [info.loc[x][info_key] for x in antecedent]
                    consequent = [info.loc[x][info_key] for x in consequent]
                
                confidence = rules.confidence
                lift = rules.lift

                print("%s => %s (support=%.4f, confidence=%.2f, lift=%.2f)" %
                      (antecedent, consequent, support, confidence, lift))
            print()

In [3]:
transactions = [
    ['beer', 'chips', 'nuts', 'olives'],
    ['beer', 'chips', 'olives'],
    ['chips', 'nuts' ],
    ['chips', 'olives'],
    ['beer', 'chips', 'olives'],
    ['chips'],
    ['nuts', 'olives'],
    ['beer', 'nuts'],
    ['beer', 'chips', 'olives'], 
    ['beer', 'nuts', 'olives'], 
    ['nuts', 'chips', 'olives'], 
    ['beer', 'olives'],
    ['chips', 'olives'], 
    ['beer', 'chips', 'olives'],
    ['beer'],
    ['chips', 'nuts' ],
    ['chips', 'olives'],
    ['chips', 'beer' ],
    ['beer', 'olives'],
    ['beer', 'chips', 'nuts', 'olives'],
    ['beer', 'chips', 'olives']

]
results = list(apriori(transactions, min_support=0.3, min_confidence=0.75, min_lift=1.0))


In [4]:
print_apyori_output(results)

Rules involving itemset ['beer', 'olives']
['beer'] => ['olives'] (support=0.4762, confidence=0.77, lift=1.08)

Rules involving itemset ['beer', 'olives', 'chips']
['beer', 'chips'] => ['olives'] (support=0.3333, confidence=0.88, lift=1.22)



#### In the first case we can see ['beer'] => ['olives'] (support=0.4762, confidence=0.77, lift=1.08)
- The support is the times that appers something over the total then we are doing
>- Support (X) = 13 / 21 = 0.6190  // Where X = beer and Y = olives
>- Support (Y) = 15 / 21 = 0.7142
>- Supp(X ∩ Y) = 10 / 21   (saying that appers 10 times over 21) = 0.4762  
>- Confidence = support (X ∩ Y) / support (X) = 0.4762/0.6190 = 0.7693 that (≈) 0.77 
>- Lift =  support (X ∩ Y) / support(X)* support(Y) =  0.4762 / (0.6190* 0.7142) = 1.077 (≈) 1.08

We can see that the Confidence is quite big, that's saying that we have a lot of probabilty that when beer appers olives will appear too.
Also we can cleary look at lift, this is the ratio between the observed support and the  expected support if X and Y were independent, this ratio in this case is greater than 1

With this we can be sure that the values calculated with python are correct. And we can do in the same way with the other example.

# 2. Load and prepare the shopping baskets

In [5]:
# LEAVE AS-IS

# File names
INPUT_PRODUCTS = "instacart-products.csv"
INPUT_TRANSACTIONS = "instacart-transactions.csv.gz"

# Read into a dataframe
products = pd.read_csv(INPUT_PRODUCTS, delimiter=",")

# Set product_id as index, and drop column aisle_id
products = products.set_index('product_id').drop(columns=['aisle_id'])

products.head(100)

,product_name,department_id
product_id,,
1,Chocolate Sandwich Cookies,19
2,All-Seasons Salt,13
3,Robust Golden Unsweetened Oolong Tea,7
4,Smart Ones Classic Favorites Mini Rigatoni Wit...,1
5,Green Chile Anytime Sauce,13
...,...,...
96,Sprinklez Confetti Fun Organic Toppings,13
97,Organic Chamomile Lemon Tea,7
98,2% Yellow American Cheese,16


## 2.1. Select by department

In [6]:
# LEAVE AS-IS

DEPT_BAKERY = 3
DEPT_VEGGIES = 4
DEPT_ALCOHOL = 5
DEPT_WORLD = 6
DEPT_DRINKS = 7
DEPT_PETS = 8
DEPT_PHARMACY = 11
DEPT_CLEANING = 17
DEPT_BABIES = 18

In [7]:
departamentos = {
    3: "DEPT_BAKERY",
    4: "DEPT_VEGGIES",
    5: "DEPT_ALCOHOL",
    6: "DEPT_WORLD",
    7: "DEPT_DRINKS",
    8: "DEPT_PETS",
    11: "DEPT_PHARMACY",
    17: "DEPT_CLEANING",
    18: "DEPT_BABIES"
}

In [8]:
def select_from_departments(products, products_ids, departments_ids):
    selected_ids = []
    for product_id in products_ids:
        if products.loc[product_id].department_id in departments_ids:
            selected_ids.append(product_id)
    
    
    return selected_ids

In [13]:
def test(products, products_ids, departments_ids):
    print("-- Test case --")
    print(products_ids)
    print("\n-- Input products --")
    for product in products_ids:
        print(product, products.loc[product].product_name, "(dept",products.loc[product].department_id,")")
    selected = select_from_departments(products, products_ids, departments_ids)
    print("\n-- Selected products --")
    for product in selected:
        print(product, products.loc[product].product_name, "(dept",products.loc[product].department_id,")")

In [14]:
# TEST 1
ids = [22, 26, 45, 54, 57, 71, 111, 112]
deps = [DEPT_BAKERY, DEPT_CLEANING]
test(products, ids, deps)

-- Test case --
[22, 26, 45, 54, 57, 71, 111, 112]

-- Input products --
22 Fresh Breath Oral Rinse Mild Mint (dept 11 )
26 Fancy Feast Trout Feast Flaked Wet Cat Food (dept 8 )
45 European Cucumber (dept 4 )
54 24/7 Performance Cat Litter (dept 8 )
57 Flat Toothpicks (dept 17 )
71 Ultra 7 Inch Polypropylene Traditional Plates (dept 17 )
111 Fabric Softener, Geranium Scent (dept 17 )
112 Hot Tomatillo Salsa (dept 13 )

-- Selected products --
57 Flat Toothpicks (dept 17 )
71 Ultra 7 Inch Polypropylene Traditional Plates (dept 17 )
111 Fabric Softener, Geranium Scent (dept 17 )


In [17]:
# TEST 2
deps = [DEPT_VEGGIES, DEPT_WORLD]
test(products, ids, deps)

-- Test case --
[22, 26, 45, 54, 57, 71, 111, 112]

-- Input products --
22 Fresh Breath Oral Rinse Mild Mint (dept 11 )
26 Fancy Feast Trout Feast Flaked Wet Cat Food (dept 8 )
45 European Cucumber (dept 4 )
54 24/7 Performance Cat Litter (dept 8 )
57 Flat Toothpicks (dept 17 )
71 Ultra 7 Inch Polypropylene Traditional Plates (dept 17 )
111 Fabric Softener, Geranium Scent (dept 17 )
112 Hot Tomatillo Salsa (dept 13 )

-- Selected products --
45 European Cucumber (dept 4 )


In [19]:
# TEST 3
deps = [DEPT_ALCOHOL, DEPT_PHARMACY]
test(products, ids, deps)

-- Test case --
[22, 26, 45, 54, 57, 71, 111, 112]

-- Input products --
22 Fresh Breath Oral Rinse Mild Mint (dept 11 )
26 Fancy Feast Trout Feast Flaked Wet Cat Food (dept 8 )
45 European Cucumber (dept 4 )
54 24/7 Performance Cat Litter (dept 8 )
57 Flat Toothpicks (dept 17 )
71 Ultra 7 Inch Polypropylene Traditional Plates (dept 17 )
111 Fabric Softener, Geranium Scent (dept 17 )
112 Hot Tomatillo Salsa (dept 13 )

-- Selected products --
22 Fresh Breath Oral Rinse Mild Mint (dept 11 )


## 2.2. Read and filter transactions

In [22]:
def add_transactions_from_deps(reader,products,departments):
    # Iterate through the CSV file
    transactions = []
    transactions_stored = 0
    transactions_read = 0
    for row in reader:
        items = [int(x) for x in row]

        selected_items = select_from_departments(products, items, departments)
        

        if selected_items != []:  # Check if its not empty
            transactions.append(selected_items)
            transactions_stored += 1
            
        transactions_read +=1
            
        if transactions_read % 1000 == 0:  # Just print each 1000 transictions
            print("Transactions Read:" ,transactions_read, "  Transactions Stored:", transactions_stored)
            
        if transactions_stored == 5000:  # To stop at exactly 5000
            print("Transactions Read:" ,transactions_read, "  Transactions Stored:", transactions_stored)
            break
            
    return transactions

In [23]:
transactions = []
departments =[DEPT_CLEANING]
# Open a compressed file
with gzip.open(INPUT_TRANSACTIONS, "rt") as inputfile:
    
    # Create a CSV reader
    reader = csv.reader(inputfile, delimiter=",")
    # Iterate through the CSV file
    transactions = add_transactions_from_deps(reader,products,[DEPT_CLEANING])


Transactions Read: 1000   Transactions Stored: 158
Transactions Read: 2000   Transactions Stored: 311
Transactions Read: 3000   Transactions Stored: 460
Transactions Read: 4000   Transactions Stored: 598
Transactions Read: 5000   Transactions Stored: 745
Transactions Read: 6000   Transactions Stored: 902
Transactions Read: 7000   Transactions Stored: 1067
Transactions Read: 8000   Transactions Stored: 1206
Transactions Read: 9000   Transactions Stored: 1373
Transactions Read: 10000   Transactions Stored: 1515
Transactions Read: 11000   Transactions Stored: 1670
Transactions Read: 12000   Transactions Stored: 1807
Transactions Read: 13000   Transactions Stored: 1951
Transactions Read: 14000   Transactions Stored: 2102
Transactions Read: 15000   Transactions Stored: 2245
Transactions Read: 16000   Transactions Stored: 2384
Transactions Read: 17000   Transactions Stored: 2543
Transactions Read: 18000   Transactions Stored: 2692
Transactions Read: 19000   Transactions Stored: 2840
Transact

## 2.3. Extract association rules and comment on them (DEPT_CLEANING)

In [15]:
results = list(apriori(transactions, min_support=0.001, min_confidence=0.2, min_lift=1))
print_apyori_output(results, products, 'product_name')


Rules involving itemset [47865, 5047]
['Easy Open TabsBags'] => ['Quart Storage Bags'] (support=0.0012, confidence=0.23, lift=38.46)

Rules involving itemset [30290, 8021]
['100% Recycled Bath Tissue Rolls'] => ['100% Recycled Paper Towels'] (support=0.0016, confidence=0.20, lift=3.69)

Rules involving itemset [37357, 8021]
['Natural Laundry Detergent, Free & Clear 33'] => ['100% Recycled Paper Towels'] (support=0.0010, confidence=0.21, lift=3.84)

Rules involving itemset [31801, 21653]
['Compostable Forks'] => ['9 Inch Plates'] (support=0.0010, confidence=0.25, lift=35.71)

Rules involving itemset [41387, 21653]
['Compostable Forks'] => ['Plastic Spoons'] (support=0.0016, confidence=0.40, lift=90.91)
['Plastic Spoons'] => ['Compostable Forks'] (support=0.0016, confidence=0.36, lift=90.91)



#### Comment:
As we can see here we have some relations, this means that it is frequent to shoppers that buy the antecedent to buy also the consequence, in this case we put 0.2 of confidence to not get many results.
- Based on the observed trend of products being related in customer transactions, I would recommend the Cleaning Department to consider storing these related products near each other on the same shelving. This arrangement can encourage customers to purchase pairs of related products, making their shopping experience more convenient and potentially increasing sales.

Analizing the associations we can see that:

> The **first** association,we can see that they are two type of bags
The **second** association, we can see that there are two hygienic items can are bought togheter
The **third** association, they are two disposable products that are normally used together
The **fourth** and fifth association, could be a bidirectional asssociation, were its really common to buy them together in both orders. We can see that that the confidence is high here. The data shows that transactions tend to associate or include **products that are similar** or related to each other.



## 2.4. Extract association rules and comment on them (other departments)

In [16]:
transactions = []
departments =[DEPT_ALCOHOL, DEPT_DRINKS]

with gzip.open(INPUT_TRANSACTIONS, "rt") as inputfile:  
    # Create a CSV reader
    reader = csv.reader(inputfile, delimiter=",")
    transactions = add_transactions_from_deps(reader,products,departments)

Transactions Read: 1000 Transactions Stored: 453
Transactions Read: 2000 Transactions Stored: 901
Transactions Read: 3000 Transactions Stored: 1395
Transactions Read: 4000 Transactions Stored: 1872
Transactions Read: 5000 Transactions Stored: 2326
Transactions Read: 6000 Transactions Stored: 2785
Transactions Read: 7000 Transactions Stored: 3256
Transactions Read: 8000 Transactions Stored: 3743
Transactions Read: 9000 Transactions Stored: 4190
Transactions Read: 10000 Transactions Stored: 4667
Transactions Read: 10644 Transactions Stored: 5000


In [17]:
results = list(apriori(transactions, min_support=0.005, min_confidence=0.2, min_lift=1))
print_apyori_output(results, products, 'product_name')

Rules involving itemset [44632, 14947]
['Pure Sparkling Water'] => ['Sparkling Water Grapefruit'] (support=0.0058, confidence=0.26, lift=5.02)

Rules involving itemset [21709, 35221]
['Sparkling Lemon Water'] => ['Lime Sparkling Water'] (support=0.0056, confidence=0.29, lift=8.59)

Rules involving itemset [44632, 21709]
['Sparkling Lemon Water'] => ['Sparkling Water Grapefruit'] (support=0.0060, confidence=0.31, lift=5.95)

Rules involving itemset [44632, 35221]
['Lime Sparkling Water'] => ['Sparkling Water Grapefruit'] (support=0.0100, confidence=0.30, lift=5.72)



>As I mentioned earlier, associations tend to link items that share similarities. In this case, it's evident that all the connections involve various kinds of flavored sparkling water.

>Furthermore, we can observe that the confidence level falls within the range of 0.25 to 0.30



In [18]:
# First we will select diferent departments
transactions = []
departments =[DEPT_ALCOHOL, DEPT_VEGGIES,DEPT_BAKERY, DEPT_WORLD,DEPT_DRINKS]

with gzip.open(INPUT_TRANSACTIONS, "rt") as inputfile:
    
    # Create a CSV reader
    reader = csv.reader(inputfile, delimiter=",")
    transactions = add_transactions_from_deps(reader,products,departments)
 


Transactions Read: 1000 Transactions Stored: 915
Transactions Read: 2000 Transactions Stored: 1810
Transactions Read: 3000 Transactions Stored: 2704
Transactions Read: 4000 Transactions Stored: 3614
Transactions Read: 5000 Transactions Stored: 4513
Transactions Read: 5555 Transactions Stored: 5000


In [19]:
results = list(apriori(transactions, min_support=0.0025, min_confidence=0.2, min_lift=0.5))
different_departments = []
for res in results:   
    departments_id = []
    product_id =[]
    my_frozenset = res.items
    for value in my_frozenset:
        departments_id.append(products.loc[value].department_id)
        product_id.append(value)
    #cambiar
    unique_departments = set(departments_id)
    if len(unique_departments) == len(departments_id):
        #print(departments_id)
        different_departments.append(product_id)
        
print("TRANSACTIONS WITH PRODUCTS FROM DIFERENT DEPARTMENTS:")
for item in different_departments:
    print("   -",products.loc[item[0]].product_name, "from", departamentos[products.loc[item[0]].department_id]
          ,"and" ,products.loc[item[1]].product_name, "from",departamentos[products.loc[item[1]].department_id])

TRANSACTIONS WITH PRODUCTS FROM DIFERENT DEPARTMENTS:
   - Bag of Organic Bananas from DEPT_VEGGIES and 100% Whole Wheat Bread from DEPT_BAKERY
   - Banana from DEPT_VEGGIES and 100% Whole Wheat Bread from DEPT_BAKERY
   - Banana from DEPT_VEGGIES and Sparkling Lemon Water from DEPT_DRINKS
   - Banana from DEPT_VEGGIES and Lime Sparkling Water from DEPT_DRINKS


>With this code we use the Apriori algorithm to mine association rules from a dataset of transactions. The goal is to find patterns or associations between products that appear together frequently in these transactions.
The code essentially finds and prints transactions where customers buy products from different departments together, helping identify potentially interesting patterns or associations between products that might not be immediately obvious but have some significance.

<font size="+1" color="#003300">We hereby declare that, except for the code provided by the course instructors, all of my code, report, and figures were produced by myself.</font>